# Initialization of plate generation for synthesis


Ho importato le librerie che mi servono

In [25]:
import numpy as np
import random
import os

Creo la cartella dove salvare l'inizializzazione degli esperimenti

In [26]:
data_folder = "data_syn"
os.makedirs(data_folder, exist_ok=True)  # crea se non esiste

Definisco le concentrazioni da testare con variabili liste


> Aggiungi citazione




In [27]:
mg_gluta=[6, 7, 8, 9, 10, 11, 12, 13, 14]
peg=[3, 4.5, 6, 7.5]
clpx=[0, 50, 100, 150, 200, 250, 300, 350, 400]
clpp=[0, 50, 100, 150, 200, 250, 300, 350, 400]
atp=[0, 1, 2, 3, 4, 5, 6, 7, 8]
dna=[0, 3]
lists=[mg_gluta,peg,clpx,clpp,atp]

Definisco i control samples


In [28]:
ref_syn=[mg_gluta[-1],
         peg[-1],
         clpx[-1],
         clpp[-1],
         atp[-1],
         dna[-1]
]
autofl_syn=[mg_gluta[-1],
         peg[-1],
         clpx[-1],
         clpp[-1],
         atp[-1],
         dna[0]
]


conversione ad array e creazione di quello di controllo

In [29]:
ref_syn=np.reshape(ref_syn,(1,6))
autofl_syn=np.reshape(autofl_syn,(1,6))

ctrl_syn=np.concatenate((ref_syn,autofl_syn),axis=0)


Definisco tutti massimi tranne uno minimo per la sintesi. Sono combinazioni che voglio testare all'inizio

In [30]:
boundary_max_syn=ref_syn
this_sample=[mg_gluta[0],
         peg[-1],
         clpx[-1],
         clpp[-1],
         atp[-1],
         dna[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_max_syn=np.concatenate((boundary_max_syn,this_sample),axis=0)

In [31]:
this_sample=[mg_gluta[-1],
         peg[0],
         clpx[-1],
         clpp[-1],
         atp[-1],
         dna[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_max_syn=np.concatenate((boundary_max_syn,this_sample),axis=0)


In [32]:
this_sample=[mg_gluta[-1],
         peg[-1],
         clpx[0],
         clpp[-1],
         atp[-1],
         dna[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_max_syn=np.concatenate((boundary_max_syn,this_sample),axis=0)

In [33]:
this_sample=[mg_gluta[-1],
         peg[-1],
         clpx[-1],
         clpp[0],
         atp[-1],
         dna[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_max_syn=np.concatenate((boundary_max_syn,this_sample),axis=0)


In [34]:
this_sample=[mg_gluta[-1],
         peg[-1],
         clpx[-1],
         clpp[-1],
         atp[0],
         dna[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_max_syn=np.concatenate((boundary_max_syn,this_sample),axis=0)


In [35]:
print(boundary_max_syn.shape)

(6, 6)


Ora definisco tutti i minimi tranne uno per la sintesi, compreso quello con tutti i valori al minimo


In [36]:
this_sample=[mg_gluta[0],
         peg[0],
         clpx[0],
         clpp[0],
         atp[0],
         dna[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_min_syn=this_sample

In [37]:
this_sample=[mg_gluta[-1],
         peg[0],
         clpx[0],
         clpp[0],
         atp[0],
         dna[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_min_syn=np.concatenate((boundary_min_syn,this_sample),axis=0)

In [38]:
this_sample=[mg_gluta[0],
         peg[-1],
         clpx[0],
         clpp[0],
         atp[0],
         dna[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_min_syn=np.concatenate((boundary_min_syn,this_sample),axis=0)

In [39]:
this_sample=[mg_gluta[0],
         peg[0],
         clpx[-1],
         clpp[0],
         atp[0],
         dna[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_min_syn=np.concatenate((boundary_min_syn,this_sample),axis=0)

In [40]:
this_sample=[mg_gluta[0],
         peg[0],
         clpx[0],
         clpp[-1],
         atp[0],
         dna[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_min_syn=np.concatenate((boundary_min_syn,this_sample),axis=0)

In [41]:
this_sample=[mg_gluta[0],
         peg[0],
         clpx[0],
         clpp[0],
         atp[-1],
         dna[-1]]
this_sample=np.reshape(this_sample,(1,6))
boundary_min_syn=np.concatenate((boundary_min_syn,this_sample),axis=0)


In [42]:
print(boundary_min_syn.shape)

(6, 6)


Ora riempiamo il resto della lista di esperimenti iniziali da fare randomicamente

In [43]:
initial_array_syn=np.concatenate((ctrl_syn,boundary_min_syn,boundary_max_syn[1:]),axis=0) #il primo del boundary_max_syn coincide col reference array
print(initial_array_syn.shape)

(13, 6)


Funzione per vedere se una combinazione è già stata esplorata


In [44]:
def present_in_array_index(new_sample, array):
    """
    Verify if a sample is present in an array.
    """
    if array is None:
        return(False, -1)
    present = False
    new_sample = np.reshape(np.array(new_sample), (1,6))
    for i in range(array.shape[0]):
        if np.array_equiv(array[i,:],new_sample):
            present = True
            break
    return(present, i)

Genero nuovi sample randomici

In [45]:
def generate_random_grid(array_to_avoid, sample_size = 20): #potremmo fare alla prima iterazione 20 e poi le altre di 10 in 10
    active_learning_array = array_to_avoid #array contenente tutte le combinazioni testate e poi quelle che verranno testate
    if array_to_avoid is None:
        answerSize = 0
    else:
        answerSize = active_learning_array.shape[0] #tiene conto delle scelte già fatte. Contatore dei samples già decisi
    # Reagents not allowed to vary
    dna_fixed = dna[1]

    while answerSize < sample_size:
      indexes= tuple(np.random.randint(0, len(lst)) for lst in lists)
      this_sample=[lists[0][indexes[0]],
                  lists[1][indexes[1]],
                  lists[2][indexes[2]],
                  lists[3][indexes[3]],
                  lists[4][indexes[4]],
                  dna_fixed]
      this_sample=np.reshape(this_sample,(1,6))
      if not present_in_array_index(this_sample, active_learning_array)[0]:
        answerSize = answerSize + 1
      if active_learning_array is None:
        active_learning_array = this_sample
      else:
          active_learning_array = np.concatenate((active_learning_array, this_sample), axis = 0)
    return(active_learning_array)

In [46]:
active_learning_array=generate_random_grid(initial_array_syn, sample_size = 15)
print(active_learning_array)

[[ 14.    7.5 400.  400.    8.    3. ]
 [ 14.    7.5 400.  400.    8.    0. ]
 [  6.    3.    0.    0.    0.    3. ]
 [ 14.    3.    0.    0.    0.    3. ]
 [  6.    7.5   0.    0.    0.    3. ]
 [  6.    3.  400.    0.    0.    3. ]
 [  6.    3.    0.  400.    0.    3. ]
 [  6.    3.    0.    0.    8.    3. ]
 [  6.    7.5 400.  400.    8.    3. ]
 [ 14.    3.  400.  400.    8.    3. ]
 [ 14.    7.5   0.  400.    8.    3. ]
 [ 14.    7.5 400.    0.    8.    3. ]
 [ 14.    7.5 400.  400.    0.    3. ]
 [ 11.    6.  400.  400.    0.    3. ]
 [  6.    4.5 400.  350.    5.    3. ]]


Salvo i dati


In [47]:
headers="mg_gluta; peg8000; ClpX; ClpP; ATP; DNA"
np.savetxt(f"{data_folder}/initial_plate_syn_nolabel.csv", active_learning_array, delimiter=";", header=headers, fmt='%.2f', comments="")